##Datascience Homework 3##
Author: Aedan Wells

Date: 10/13/22

Brief: 

This entry completes all portions asked:


* Completes the functions get_market_cap and get_digit_patterns, both returning lists of dictionaries for the data asked
* Completes text queries/parsing using re library, raw strings and in re.VERBOSE mode.
* No hard queries (if string1 in string2, etc) are ran

Notes:
* I chose not to split this up into more functions. There are some elements that could be removed for ease of viewing, however, there are no elements that are repeated many times in these functions. 
* Secondly, It was difficult for me to nail down what a slice was to make sure the values were correct. I am using the python definition of a slice (string.start():string.end()) which in our case is always the start of the first ddx and the start of the second ddx+3. 


In [ ]:
import re

In [ ]:
#Reads in a file name and breaks the text into a full dictionary
#param filename is the file being parsed for the market cap
#return a dictionary with the market cap data
def get_market_cap(filename):
  f_market = open(filename, 'r')
  #read one line in to avoid header
  next(f_market)
  lines = f_market.readlines()

  prev_year = 0
  company_list = []
  #for each line in the document
  for line in lines:
    company_dict = {}
    #create regex pattern
    pattern = r"""
    (^[A-Z]\w+(?:\s[A-Z][a-z]*){0,}) #the first word (capitalized) and then 0 or more capitalized words. This will get all company names
    .(had|has|reached)               #the company words up to the phrases had, has, or reached.
    (.+?(?=\$))                      #all words after had/has/reached but before the $ of the money
    (\$)                             #the $ and just that
    ([0-9]*[.]{0,1}[0-9]*)           #the numbers right after the dollar sign, any number of 0-9 values followed by 0 or 1 period and any numbers after
    (\s\w+)                          #the word after the numbers which will be the unit of the value
    (\s\w+)?                         #select the next word after unit IF it exists. looking for words like in, now, today, '', or around
    (\s\w+)?                         #select the word after time context word IF it exists. Looking for a year number or possibility of 'same time' marker
    (.*)?                            #Grabs the rest of the letters that could have things like 'same time'
    """
    #breaks down the string using the pattern above
    sentence = re.findall(pattern, line, re.VERBOSE)
    #grab company name
    company_dict['company'] = sentence[0][0]
    #grab valuation of the company
    company_dict['value'] = float(sentence[0][4])
    #grabs the unit for the valuation
    company_dict['unit'] = sentence[0][5][1:]
    #break down the time/year value
    #in has year right after
    if sentence[0][6] == ' in':
      company_dict['year'] = int(sentence[0][7][1:])
    #if word after unit is now, today, or empty, it is set in 2022
    elif sentence[0][6] == ' now' or sentence[0][6] == ' today' or sentence[0][6] == '':
      company_dict['year'] = 2022
    elif sentence[0][6] == ' around':
      #if word after around is year, put that in. else, check to see if it is a reference to same time
      if sentence[0][7][1:].isnumeric():
        company_dict['year'] = int(sentence[0][7][1:])
      else:
        if re.search(r"same\stime", sentence[0][8], re.VERBOSE):
          company_dict['year'] = prev_year
    #check if reference to same time is in the sentence and work with that
    elif re.search(r"same\stime", line, re.VERBOSE):
      company_dict['year'] = prev_year
    #add the company to the list, set the previous sentence year to this year.
    company_list.append(company_dict)
    prev_year = company_dict['year']
  return company_list

In [ ]:
#takes in a file with a long string od numbers and returns a dictioanry with many subsequences
#and it's locations in the page
#param filename is the file of a long sequence of digits
#returns a dictionary with each d and x combo that has substrings in the form ddx...ddx
def get_digit_patterns(filename):
  pi_file = open(filename, 'r')
  pi_digits = pi_file.read()
  substrings = []
  #go through all possible ddx digits
  for d in range(0, 10):
    for x in range(0, 10):
      #set some counters and holders
      last_iter_loc = 0
      last_iter_counter = 0
      #pattern group is the ddx number, nothing more
      number = r"("+ str(d) + str(d) + str(x) + ")"
      #for each instance of the number
      for item in re.finditer(number, pi_digits, re.M & re.VERBOSE):
        #increase the counter by 1
        last_iter_counter += 1
        #hold the last iter temporarily and grab the index
        tmp_iter_loc = last_iter_loc
        last_iter_loc = item.start()
        #if we have seen this number more than once
        if last_iter_counter == 2:
          #preserves getting no duplicates, if there are 3 ddx's in the list at index 1, 50, 100 it will only grab the slice 1:50 and give that info
          last_iter_counter = 0
          #print the value with locations/length and add the value to the dictionary
          print("found " + number[1:-1] + " in slice " + str(tmp_iter_loc) +":"+ str(last_iter_loc + 3) +" of length " + str(last_iter_loc - tmp_iter_loc + 3))
          instance_dict = {}
          instance_dict['d'] = d
          instance_dict['x'] = x
          instance_dict['start'] = tmp_iter_loc
          instance_dict['end'] = last_iter_loc + 3
          instance_dict['length'] = last_iter_loc - tmp_iter_loc + 3
          substrings.append(instance_dict)
  return substrings

In [ ]:
CMP_DICT = get_market_cap('market-cap-history.txt')
DGT_DICT = get_digit_patterns('digits_of_pi.txt')

found 000 in slice 612:874 of length 262
found 001 in slice 366:994 of length 628
found 111 in slice 155:1004 of length 849
found 113 in slice 368:775 of length 407
found 117 in slice 94:159 of length 65
found 117 in slice 434:447 of length 13
found 118 in slice 452:864 of length 412
found 119 in slice 502:1005 of length 503
found 223 in slice 136:842 of length 706
found 224 in slice 544:676 of length 132
found 227 in slice 492:702 of length 210
found 330 in slice 370:411 of length 41
found 334 in slice 218:849 of length 631
found 336 in slice 512:519 of length 7
found 440 in slice 520:670 of length 150
found 445 in slice 59:822 of length 763
found 446 in slice 126:187 of length 61
found 446 in slice 220:464 of length 244
found 661 in slice 987:1002 of length 15
found 664 in slice 118:283 of length 165
found 665 in slice 214:386 of length 172
found 669 in slice 261:605 of length 344
found 771 in slice 636:758 of length 122
found 778 in slice 632:647 of length 15
found 778 in slice 965: